# Lesson 3: Building an Agent Reasoning Loop

In [44]:
from custom_utils.helper import get_openai_api_key, print_response

OPENAI_API_KEY = get_openai_api_key()

In [45]:
import nest_asyncio

nest_asyncio.apply()

## Setup the Query Tools

In [46]:
from custom_utils.utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("../resources/metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [47]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [48]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [49]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) and competitive analysis. The Architect designs the technical specifications and system architecture. The Project Manager breaks down the project into tasks and assigns them to Engineers. The Engineer develops the code based on the specifications. The QA Engineer generates unit tests and reviews the code for bugs to ensure high-quality software.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each 

In [50]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: ../resources/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-10-29
last_modified_date: 2024-10-29

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tas

In [51]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset consisting of 70 representative examples of software development tasks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset consisting of 70 representative examples of software development tasks.


In [52]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": null}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.


## Lower-Level: Debuggability and Control

In [53]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [54]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [55]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT are the Architect and the Product Manager.


In [56]:
completed_steps = agent.get_completed_steps(task.task_id)
print_response(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 161bd13a-b1da-4f50-afda-727aefa66ec4: 1

The agent roles in MetaGPT are the Architect and the Product Manager.


In [57]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 161bd13a-b1da-4f50-afda-727aefa66ec4: 1


TaskStep(task_id='161bd13a-b1da-4f50-afda-727aefa66ec4', step_id='f3069931-e83a-43f1-a0ca-87e860836658', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [58]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and access messages from other agents transparently. Additionally, agents can subscribe to relevant information based on their role profiles, ensuring efficient communication. This mechanism helps in avoiding information overload and distractions, allowing agents to focus on task-related information and dependencies. The global message pool and subscription mechanism in MetaGPT streamline communication, ensuring efficiency and relevance of shared information while filtering out irrelevant contexts to enhance utility.


In [59]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and access messages from other agents transparently. Additionally, agents can subscribe to relevant information based on their role profiles, ensuring efficient communication. This mechanism helps in avoiding information overload and distractions, allowing agents to focus on task-related information and dependencies. The global message pool and subscription mechanism in MetaGPT streamline communication, ensuring efficiency and relevance of shared information while filtering out irrelevant contexts to enhance utility.
True


In [60]:
response = agent.finalize_response(task.task_id)

In [61]:
print(str(response))

Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and access messages from other agents transparently. Additionally, agents can subscribe to relevant information based on their role profiles, ensuring efficient communication. This mechanism helps in avoiding information overload and distractions, allowing agents to focus on task-related information and dependencies. The global message pool and subscription mechanism in MetaGPT streamline communication, ensuring efficiency and relevance of shared information while filtering out irrelevant contexts to enhance utility.
